# Test Long Context RAG Answer Question

This notebook tests the `_answer_question` functionality in the `LongContextRAG` class.

In [ ]:
import os
import sys
from pathlib import Path
import tempfile
from loguru import logger
from autocoder.rag.long_context_rag import LongContextRAG
from autocoder.rag.rag_config import RagConfig
from autocoder.rag.cache.simple_cache import AutoCoderRAGAsyncUpdateQueue
from autocoder.rag.variable_holder import VariableHolder
from tokenizers import Tokenizer

# Setup tokenizer
VariableHolder.TOKENIZER_PATH = "/Users/allwefantasy/Downloads/tokenizer.json"
VariableHolder.TOKENIZER_MODEL = Tokenizer.from_file(VariableHolder.TOKENIZER_PATH)

In [ ]:
# Create test files and directory
test_dir = tempfile.mkdtemp()
print(f"Created test directory: {test_dir}")

# Create a test Python file
test_file = os.path.join(test_dir, "test_code.py")
with open(test_file, "w") as f:
    f.write("""
def calculate_sum(a: int, b: int) -> int:
    """Calculate the sum of two integers."""
    return a + b

def calculate_product(a: int, b: int) -> int:
    """Calculate the product of two integers."""
    return a * b
    """)

In [ ]:
# Initialize RAG components
config = RagConfig(
    model="gpt-4-1106-preview",
    path=test_dir,
    required_exts=[".py"],
    cache_type="simple"
)

rag = LongContextRAG(config)

# Test questions
test_questions = [
    "What does the calculate_sum function do?",
    "Show me all the functions that work with integers",
    "What's the return type of calculate_product?"
]

# Test answers
for question in test_questions:
    print(f"\nQuestion: {question}")
    answer = rag._answer_question(question)
    print(f"Answer: {answer}")

In [ ]:
# Clean up
import shutil
shutil.rmtree(test_dir)
print(f"Cleaned up test directory: {test_dir}")